In [1]:
import git
from pathlib import Path
import os

ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

DATASET = "coco"
FINAL_DATA_NAME = 'approx1e5-toy-coco-cropped-allObjects-fourier' # + channel
CONSTANT_SAMPLE_SIZE = int(1e5)
RAW_DATA_SUFFIX = None
SAVE_DF = False

data_dir = os.path.join(ROOT_DIR, 'dataset-preparation', 'raw-data', DATASET, "toy-coco-cropped-allObjects")
file_list = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(data_dir)
BATCH_NUM = None
file_names

def npz_opener_pil(path):
    # Load image as grayscale
    image = Image.open(path).convert('RGB')
    array_data = np.array(image)
    return array_data

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/SSD/hbmv/hierarchical-bayesian-model-validation/dataset-preparation/raw-data/coco/toy-coco-cropped-allObjects'

In [16]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from transform import *
os.chdir(os.path.join(ROOT_DIR, "dataset-preparation"))
freq_df = pd.read_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"), index_col= ["dataset", "transform", "group"])
npz_opener = npz_opener_pil

In [ ]:
# file_list = [os.path.join(data_dir, f"full-{RAW_DATA_SUFFIX}", filename) for filename in os.listdir(data_dir)]
# file_names = os.listdir(os.path.join(data_dir, f"full-{RAW_DATA_SUFFIX}"))
# file_names[:5]

['test519.npz', 'test525.npz', 'test243.npz', 'test257.npz', 'test531.npz']

In [29]:
'''Assuming No batching is required. Not applicable for agriVision'''

# data_dir = os.path.join(ROOT_DIR, "raw-data", "agriVision", "full-agriVision-RGB-cleaned")

# for channel in ['red', 'blue', 'green', 'gray', 'infrared']:

#     channel_fr = convert_to_fourier_basis(data_dir, channel, debug = True)
#     pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"full-agriVision-fourier-{channel}-df.pickle"))

#     min_group, max_group = 2, max(channel_fr['band'])
#     group_data_map = dict()
#     group_data_map_size = dict()
#     for group in np.arange(min_group, max_group + 1):
#         data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
#         group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
#         group_data_map_size[group] = data.size
    
#     pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
#     pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
    

'Assuming No batching is required. Not applicable for agriVision'

In [30]:
'''To split large dataset into many batches, only needs to be run once'''
# k = 10000
# target_dir = os.path.join(ROOT_DIR, 'raw-data', 'agriVision') # Where the batch{i} folders will be created
# directorySplit(folder_dir = data_dir, target_dir = target_dir, name = RAW_DATA_SUFFIX, k = k)
# print(f"{len(file_names)//k} batches created" )

'To split large dataset into many batches, only needs to be run once'

In [7]:
# '''Show all subsets of data in raw data folder that have already been created'''
# print(''.join([x+"\n" for x in os.listdir(data_dir) if x.__contains__(RAW_DATA_SUFFIX)]))


# Fourier

In [ ]:
#Values obtained from plots in pastisFourierEDA.ipynb
STARTING_VALUE = 0.059498227389561786
ENDING_VALUE = 0.6743233008450027
MULT_FACTOR = 1.08
if BATCH_NUM is None:
    batch_dir = os.path.join(ROOT_DIR, 'dataset-preparation', "raw-data", "coco", "toy-coco-cropped-allObjects")
    # batch_dir = os.path.join(ROOT_DIR, "raw-data", "pastis", f"full-{RAW_DATA_SUFFIX}")
else:
    batch_dir = os.path.join(ROOT_DIR, "raw-data", "pastis", f"batch{BATCH_NUM}-{RAW_DATA_SUFFIX}")
splits = getSplits(STARTING_VALUE,ENDING_VALUE, MULT_FACTOR)

In [18]:
batch_dir

'/Users/michaelmurphy/Documents/GitHub/hierarchical-bayesian-model-validation/dataset-preparation/raw-data/coco/toy-coco-cropped-allObjects'

In [19]:
TRANSFORM = "fourier"
channel = "red"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = np.mean(channel_fr[(channel_fr['band'] == group)]['magnitude_endpoints'].iloc[0])
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))

if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()


100%|██████████| 4958/4958 [00:15<00:00, 309.91it/s]


[0.05949823 0.06425809 0.06939873 0.07495063 0.08094668 0.08742242
 0.09441621 0.10196951 0.11012707 0.11893723 0.12845221 0.13872839
 0.14982666 0.16181279 0.17475781 0.18873844 0.20383751 0.22014452
 0.23775608 0.25677656 0.27731869 0.29950418 0.32346452 0.34934168
 0.37728901 0.40747213 0.44006991 0.4752755  0.51329754 0.55436134
 0.59871025 0.64660707 0.69833563]


100%|██████████| 33/33 [00:06<00:00,  4.97it/s]


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,red,"(0.0, 0.059112288868834205)",89,"[6586224.0, 8800123.0, 7376467.0, 7632658.0, 8..."
1,2,red,"(0.059498227389561786, 0.06406726354240909)",14,"[-21728.918, 18565.07, 2970.421, 516.0442, -19..."
2,3,red,"(0.06442352540027595, 0.06921892635417715)",15,"[-310.95404, -32257.477, 41980.53, 18873.031, ..."
3,4,red,"(0.06954880396392522, 0.07462880146305781)",17,"[9639.089, 32434.5, -4184.5127, -14871.204, 85..."
4,5,red,"(0.0750366121574162, 0.08052940675034494)",20,"[11491.171, -24967.992, 5081.701, 23661.59, -2..."


In [20]:
del channel_fr, group_data_map, group_data_map_size

In [21]:
TRANSFORM = "fourier"
channel = "blue"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = np.mean(channel_fr[(channel_fr['band'] == group)]['magnitude_endpoints'].iloc[0])
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))

if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

100%|██████████| 4958/4958 [00:15<00:00, 312.38it/s]


[0.05949823 0.06425809 0.06939873 0.07495063 0.08094668 0.08742242
 0.09441621 0.10196951 0.11012707 0.11893723 0.12845221 0.13872839
 0.14982666 0.16181279 0.17475781 0.18873844 0.20383751 0.22014452
 0.23775608 0.25677656 0.27731869 0.29950418 0.32346452 0.34934168
 0.37728901 0.40747213 0.44006991 0.4752755  0.51329754 0.55436134
 0.59871025 0.64660707 0.69833563]


100%|██████████| 33/33 [00:09<00:00,  3.60it/s]


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,blue,"(0.0, 0.059112288868834205)",89,"[5803534.0, 10886303.0, 7226035.0, 5082238.0, ..."
1,2,blue,"(0.059498227389561786, 0.06406726354240909)",14,"[-5795.5977, 19345.197, 15168.382, 34631.016, ..."
2,3,blue,"(0.06442352540027595, 0.06921892635417715)",15,"[4466.5254, -23125.18, 38461.812, 19402.598, 4..."
3,4,blue,"(0.06954880396392522, 0.07462880146305781)",17,"[-2172.0408, 22280.883, 963.404, -7072.2437, 8..."
4,5,blue,"(0.0750366121574162, 0.08052940675034494)",20,"[10736.866, -3934.8145, 3283.3906, 14182.206, ..."


In [22]:
del channel_fr, group_data_map, group_data_map_size

In [23]:
TRANSFORM = "fourier"
channel = "green"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = np.mean(channel_fr[(channel_fr['band'] == group)]['magnitude_endpoints'].iloc[0])
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))
    
if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

100%|██████████| 4958/4958 [00:15<00:00, 317.21it/s]


[0.05949823 0.06425809 0.06939873 0.07495063 0.08094668 0.08742242
 0.09441621 0.10196951 0.11012707 0.11893723 0.12845221 0.13872839
 0.14982666 0.16181279 0.17475781 0.18873844 0.20383751 0.22014452
 0.23775608 0.25677656 0.27731869 0.29950418 0.32346452 0.34934168
 0.37728901 0.40747213 0.44006991 0.4752755  0.51329754 0.55436134
 0.59871025 0.64660707 0.69833563]


100%|██████████| 33/33 [00:06<00:00,  5.02it/s]


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,green,"(0.0, 0.059112288868834205)",89,"[5915875.0, 11256216.0, 7073392.0, 6219822.0, ..."
1,2,green,"(0.059498227389561786, 0.06406726354240909)",14,"[-5098.6753, 9701.47, -4026.4194, 21153.275, -..."
2,3,green,"(0.06442352540027595, 0.06921892635417715)",15,"[8689.545, -27552.076, 45806.918, 18584.434, -..."
3,4,green,"(0.06954880396392522, 0.07462880146305781)",17,"[-1097.225, 29547.746, 2075.454, -8324.688, 38..."
4,5,green,"(0.0750366121574162, 0.08052940675034494)",20,"[11749.738, -16956.934, -2781.6377, 19640.25, ..."


In [24]:
del channel_fr, group_data_map, group_data_map_size

In [25]:
TRANSFORM = "fourier"
channel = "gray"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = np.mean(channel_fr[(channel_fr['band'] == group)]['magnitude_endpoints'].iloc[0])
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))
    
if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

100%|██████████| 4958/4958 [00:19<00:00, 259.62it/s]


[0.05949823 0.06425809 0.06939873 0.07495063 0.08094668 0.08742242
 0.09441621 0.10196951 0.11012707 0.11893723 0.12845221 0.13872839
 0.14982666 0.16181279 0.17475781 0.18873844 0.20383751 0.22014452
 0.23775608 0.25677656 0.27731869 0.29950418 0.32346452 0.34934168
 0.37728901 0.40747213 0.44006991 0.4752755  0.51329754 0.55436134
 0.59871025 0.64660707 0.69833563]


100%|██████████| 33/33 [00:06<00:00,  4.94it/s]


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,gray,"(0.0, 0.059112288868834205)",89,"[6102844.0, 10478794.0, 7180675.0, 6511812.0, ..."
1,2,gray,"(0.059498227389561786, 0.06406726354240909)",14,"[-10148.394, 13449.215, 253.54619, 16519.154, ..."
2,3,gray,"(0.06442352540027595, 0.06921892635417715)",15,"[5517.0024, -28451.1, 43821.29, 18762.107, -17..."
3,4,gray,"(0.06954880396392522, 0.07462880146305781)",17,"[1989.4401, 29579.22, 77.36879, -10137.831, 26..."
4,5,gray,"(0.0750366121574162, 0.08052940675034494)",20,"[11555.811, -17865.22, 260.40567, 20218.049, -..."


In [26]:
del channel_fr, group_data_map, group_data_map_size

# Wavelet

In [27]:
FINAL_DATA_NAME = 'approx1e5-toy-coco-cropped-allObjects-wavelet'
if BATCH_NUM is None:
    batch_dir = os.path.join(ROOT_DIR, 'dataset-preparation', "raw-data", "coco", "toy-coco-cropped-allObjects")
    # batch_dir = os.path.join(ROOT_DIR, "raw-data", "pastis", f"full-{RAW_DATA_SUFFIX}")
else:
    batch_dir = os.path.join(ROOT_DIR, "raw-data", "pastis", f"batch{BATCH_NUM}-{RAW_DATA_SUFFIX}")

In [28]:
TRANSFORM = "wavelet"
channel = "red"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group)]['frequency'].iloc[0]
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))
                             
if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

9 layers being used


100%|██████████| 4958/4958 [00:07<00:00, 651.30it/s]


,channel,layer,frequency,orientation,data
0,red,1,0.110679,L1,"[25727.438, 34375.48, 28814.324, 29815.07, 332..."
1,red,2,0.124514,D,"[3729.5, 857.0039, 8.058594, 1847.9141, 5180.6..."
2,red,2,0.124514,H,"[216.73438, 8311.707, 1480.793, -68.28125, 536..."
3,red,2,0.124514,V,"[-6172.4844, 2319.043, -2366.254, -2065.0625, ..."
4,red,3,0.142301,D,"[-1318.875, 934.5, 1311.8906, -925.2031, 2840...."


In [29]:
del channel_wv, group_data_map, group_data_map_size

In [30]:
TRANSFORM = "wavelet"
channel = "green"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group)]['frequency'].iloc[0]
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))

if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

9 layers being used


100%|██████████| 4958/4958 [00:08<00:00, 604.84it/s]


,channel,layer,frequency,orientation,data
0,green,1,0.110679,L1,"[23108.887, 43969.594, 27630.438, 24296.18, 33..."
1,green,2,0.124514,D,"[4908.16, 269.27344, 227.76562, 1377.9297, 421..."
2,green,2,0.124514,H,"[-1528.3398, 14733.172, -241.84375, 1294.4844,..."
3,green,2,0.124514,V,"[-3526.2695, 26.398438, -2296.25, -1938.9219, ..."
4,green,3,0.142301,D,"[-1616.7344, 2283.7031, 1292.9922, -2230.7969,..."


In [31]:
del channel_wv, group_data_map, group_data_map_size

In [32]:
TRANSFORM = "wavelet"
channel = "blue"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group)]['frequency'].iloc[0]
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))
    
if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

9 layers being used


100%|██████████| 4958/4958 [00:08<00:00, 595.03it/s]


,channel,layer,frequency,orientation,data
0,blue,1,0.110679,L1,"[22670.055, 42524.62, 28226.7, 19852.492, 3185..."
1,blue,2,0.124514,D,"[2713.8047, -2485.3398, 377.98047, 1710.6875, ..."
2,blue,2,0.124514,H,"[-785.66406, 16438.379, -519.5039, 2086.1719, ..."
3,blue,2,0.124514,V,"[-1392.8672, -993.3008, -2520.0508, -1965.9141..."
4,blue,3,0.142301,D,"[-1975.6797, 1450.9609, 1306.9453, -2543.836, ..."


In [33]:
del channel_wv, group_data_map, group_data_map_size

In [34]:
TRANSFORM = "wavelet"
channel = "gray"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group)]['frequency'].iloc[0]
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))

if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

9 layers being used


100%|██████████| 4958/4958 [00:12<00:00, 411.28it/s]


,channel,layer,frequency,orientation,data
0,gray,1,0.110679,L1,"[23839.234, 40932.79, 28049.512, 25436.766, 33..."
1,gray,2,0.124514,D,"[4305.2114, 130.89323, 179.19691, 1556.2046, 4..."
2,gray,2,0.124514,H,"[-921.9193, 13006.717, 241.4233, 977.2767, 123..."
3,gray,2,0.124514,V,"[-4073.6626, 595.4215, -2342.4578, -1979.5085,..."
4,gray,3,0.142301,D,"[-1568.4623, 1785.2654, 1300.1023, -1876.0183,..."


In [35]:
del channel_wv, group_data_map, group_data_map_size